# Testing notebook
This is a testing notebook, serving no other purpose other than testing things out while developping.

In [3]:
import sys
sys.path.append("..")

from src.models import OpenAIBot
from src.prompts.game24_tot import *

In [16]:
# CoT + ToT's prompt to test the model (based on ToT only 4% success rate)

input = "1 1 4 6"
bot = OpenAIBot(model="gpt-4")

prompt = cot_prompt.format(input=input)
response = bot.request(prompt)

print(f"Prompt \n{prompt}")
print("----\n----\n")
print(f"Response \n{response}")


Prompt 
Use numbers and basic arithmetic operations (+ - * /) to obtain 24. Each step, you are only allowed to choose two of the remaining numbers to obtain a new number.
Input: 4 4 6 8
Steps:
4 + 8 = 12 (left: 4 6 12)
6 - 4 = 2 (left: 2 12)
2 * 12 = 24 (left: 24)
Answer: (6 - 4) * (4 + 8) = 24
Input: 2 9 10 12
Steps:
12 * 2 = 24 (left: 9 10 24)
10 - 9 = 1 (left: 1 24)
24 * 1 = 24 (left: 24)
Answer: (12 * 2) * (10 - 9) = 24
Input: 4 9 10 13
Steps:
13 - 10 = 3 (left: 3 4 9)
9 - 3 = 6 (left: 4 6)
4 * 6 = 24 (left: 24)
Answer: 4 * (9 - (13 - 10)) = 24
Input: 1 4 8 8
Steps:
8 / 4 = 2 (left: 1 2 8)
1 + 2 = 3 (left: 3 8)
3 * 8 = 24 (left: 24)
Answer: (1 + 8 / 4) * 8 = 24
Input: 5 5 5 9
Steps:
5 + 5 = 10 (left: 5 9 10)
10 + 5 = 15 (left: 9 15)
15 + 9 = 24 (left: 24)
Answer: ((5 + 5) + 5) + 9 = 24
Input: 1 1 4 6

----
----

Response 
Steps:
6 * 4 = 24 (left: 1 1 24)
1 + 1 = 2 (left: 2 24)
24 * 2 = 48 (left: 48)
Answer: (6 * 4) * (1 + 1) = 48

Note: It's impossible to get 24 with these numbers 